# Ejercicio dataset CIFAR-10

**Profesor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />

**Colaborador:** Sebastián Arpón <br />
**E-mail:** <rmunoz@metricarts.com> <br />

En este taller usaremos el dataset CIFAR-10 (https://www.cs.toronto.edu/~kriz/cifar.html), el cual consiste de 60000 imagenes a color de tamaño 32x32 pixels y clasificado en 10 clases. Cada clase consiste de 6000 imágenes.

Tutoriales y documentación de Tensorflow

- https://www.tensorflow.org/get_started/get_started
- https://www.tensorflow.org/api_docs/

In [ ]:
import tensorflow as tf
import numpy as np
import math
import timeit

import matplotlib.pyplot as plt
%matplotlib inline

## Cargar datasets de CIFAR-10


In [ ]:
from tools.data_utils import load_CIFAR10

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=10000):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'data/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

## Modelo de ejemplo

### Algunas datos utiles

. Nuestros dataset de imágenes es inicialmente N x H x W x C, donde
* N es numero de registros
* H es la altura de cada imagen en pixels
* W es el ancho de cada imagen en pixels
* C es el numero de canales (comunmente 3: R, G, B)



### El modelo de ejemplo en sí

El primer paso es definir la arquitectura de nuestro modelo

Aquí hay un ejemplo de una red neuronal convolucional definida en TensorFlow -- intente entender que está haciendo cada línea, recordando que cada capa está construida sobre la capa anterior. No hemos entrenado nada aún - eso vendrá después - por ahora, queremos que entienda como se configura todo.

En este ejemplo, se ven capas convolucionales 2D (Conv2d), activaciones RELU, y capas completamente conectadas (Lineales). También se ven siendo usadas la función de pérdida de Hinge y el optimizador de Adam.

Asegúrese de que entiende por qué los parámetros de la capa lineal son 5408 y 10.

### Detalles TensorFlow
En TensorFlow, así como en nuestros notebooks anteriores, primero inicializaremos nuestras variables y luego nuestro modelo de red.


In [ ]:
# clear old variables
tf.reset_default_graph()

# setup input (e.g. the data that changes every batch)
# The first dim is None, and gets sets automatically based on batch size fed in
X = tf.placeholder(tf.float32, [None, 32, 32, 3])
y = tf.placeholder(tf.int64, [None])
is_training = tf.placeholder(tf.bool)

def simple_model(X,y):
    # define our weights (e.g. init_two_layer_convnet)
    
    # setup variables
    Wconv1 = tf.get_variable("Wconv1", shape=[7, 7, 3, 32])
    bconv1 = tf.get_variable("bconv1", shape=[32])
    W1 = tf.get_variable("W1", shape=[5408, 10])
    b1 = tf.get_variable("b1", shape=[10])

    # define our graph (e.g. two_layer_convnet)
    a1 = tf.nn.conv2d(X, Wconv1, strides=[1,2,2,1], padding='VALID') + bconv1
    h1 = tf.nn.relu(a1)
    h1_flat = tf.reshape(h1,[-1,5408])
    y_out = tf.matmul(h1_flat,W1) + b1
    return y_out

y_out = simple_model(X,y)

# define our loss
total_loss = tf.losses.hinge_loss(tf.one_hot(y,10),logits=y_out)
mean_loss = tf.reduce_mean(total_loss)

# define our optimizer
optimizer = tf.train.AdamOptimizer(5e-4) # select optimizer and set learning rate
train_step = optimizer.minimize(mean_loss)

TensorFlow soporta muchos otros tipos de capas, funciones de pérdida y optimizadores - experimentará con estos a continuación. Aquí está la documentación oficial de la API para estos (si alguno de los parámetros usados arriba no fueron claros, este recurso también será útil).

* Capas, Activaciones, Funciones de Pérdida : https://www.tensorflow.org/api_guides/python/nn
* Optimizadores: https://www.tensorflow.org/api_guides/python/train#Optimizers
* BatchNorm: https://www.tensorflow.org/api_docs/python/tf/layers/batch_normalization

### Entrenando el Módelo en una Época
Mientras hemos definido un "graph" de operaciones arriba entregándole datos de entrada y computando los resultados, para ejecutar "Graphs" de TensorFlow primero debemos crear un objeto `tf.Session`. Una `tf.Session` encapsula el control y el estado de tiempo de ejecución de TensorFlow. Para más información, vea la guía de TensorFlow [Comenzando con TensorFlow](https://www.tensorflow.org/get_started/get_started).

Opcionalmente también podemos especificar el contexto de dispositivo, como por ejemplo `/cpu:0` o `/gpu:0`. Para documentación sobre este comportamiento vea [esta guía de TensorFlow](https://www.tensorflow.org/tutorials/using_gpu)

Abajo debería ver una pérdida de validación entre 0.4 y 0.6, y una precisión entre 0.30 y 0.35.

In [ ]:
def run_model(session, predict, loss_val, Xd, yd,
              epochs=1, batch_size=64, print_every=100,
              training=None, plot_losses=False):
    # have tensorflow compute accuracy
    correct_prediction = tf.equal(tf.argmax(predict,1), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None
    
    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute
    variables = [mean_loss,correct_prediction,accuracy]
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    for e in range(epochs):
        # keep track of losses and accuracy
        correct = 0
        losses = []
        # make sure we iterate over the dataset once
        for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
            # generate indicies for the batch
            start_idx = (i*batch_size)%Xd.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            # create a feed dictionary for this batch
            feed_dict = {X: Xd[idx,:],
                         y: yd[idx],
                         is_training: training_now }
            # get batch size
            actual_batch_size = yd[idx].shape[0]
            
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
            loss, corr, _ = session.run(variables,feed_dict=feed_dict)
            
            # aggregate performance stats
            losses.append(loss*actual_batch_size)
            correct += np.sum(corr)
            
            # print every now and then
            if training_now and (iter_cnt % print_every) == 0:
                print("Iteration {0}: with minibatch training loss = {1:.3g} and accuracy of {2:.2g}"\
                      .format(iter_cnt,loss,np.sum(corr)/actual_batch_size))
            iter_cnt += 1
        total_correct = correct/Xd.shape[0]
        total_loss = np.sum(losses)/Xd.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}"\
              .format(total_loss,total_correct,e+1))
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    return total_loss,total_correct

with tf.Session() as sess:
    with tf.device("/cpu:0"): #"/cpu:0" or "/gpu:0" 
        sess.run(tf.global_variables_initializer())
        print('Training')
        run_model(sess,y_out,mean_loss,X_train,y_train,1,64,100,train_step,True)
        print('Validation')
        run_model(sess,y_out,mean_loss,X_val,y_val,1,64)

## Entrenando un modelo específico
En esta sección vamos a especificar un modelo para que usted construya. El objetivo no es obtener un buen rendimiento (eso será después), si no que en vez acostumbrarse y familiarizarse con entender la documentación de TensorFlow y la configuración de su propio modelo. 

Usando el código provisto arriba como guía, y usando la documentación de TensorFlow apropiada, especifique un modelo con la siguiente arquitectura:

* 7x7 Convolutional Layer with 32 filters and stride of 1
* ReLU Activation Layer
* Spatial Batch Normalization Layer (trainable parameters, with scale and centering)
* 2x2 Max Pooling layer with a stride of 2
* Affine layer with 1024 output units
* ReLU Activation Layer
* Affine layer from 1024 input units to 10 outputs



In [ ]:
# clear old variables
tf.reset_default_graph()

# define our input (e.g. the data that changes every batch)
# The first dim is None, and gets sets automatically based on batch size fed in
X = tf.placeholder(tf.float32, [None, 32, 32, 3])
y = tf.placeholder(tf.int64, [None])
is_training = tf.placeholder(tf.bool)

# define model
def complex_model(X,y,is_training):
    
    # Set up variables
    Wconv1 = tf.get_variable("Wconv1",shape=[7,7,3,32])
    bconv1 = tf.get_variable("bconv1",shape=[32])
    Waffine1 = tf.get_variable("Waffine1", shape=[5408,1024])
    baffine1 = tf.get_variable("baffine1", shape=[1024])
    Waffine2 = tf.get_variable("Waffine2", shape=[1024,10])
    baffine2 = tf.get_variable("baffine2", shape=[10])
    
    # Define model
    conv1act = tf.nn.conv2d(X, Wconv1, strides=[1,1,1,1], padding='VALID') + bconv1
    relu1act = tf.nn.relu(conv1act)
    batchnorm1act = tf.layers.batch_normalization(relu1act, training=is_training)  # Using the tf.layers batch norm as its easier
    pool1act = tf.nn.max_pool(batchnorm1act, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    flatten1 = tf.reshape(pool1act,[-1,5408])
    affine1act = tf.matmul(flatten1, Waffine1) + baffine1
    relu2act = tf.nn.relu(affine1act)
    y_out = tf.matmul(relu2act, Waffine2) + baffine2
    
    return y_out

y_out = complex_model(X,y,is_training)

Para asegurarse de que está haciendo el ejercicio de manera correcta, use la siguiente herramienta para revisar la dimensionalidad de su salida (debería ser 64 x 10, debido a que nuestros "batches" tiene un tamaño de 64 y el output de la "affine layer" final debería ser 10, correspondiendo a nuestras 10 clases):


In [ ]:
# Now we're going to feed a random batch into the model 
# and make sure the output is the right size
x = np.random.randn(64, 32, 32,3)
with tf.Session() as sess:
    with tf.device("/cpu:0"): #"/cpu:0" or "/gpu:0"
        tf.global_variables_initializer().run()

        ans = sess.run(y_out,feed_dict={X:x,is_training:True})
        %timeit sess.run(y_out,feed_dict={X:x,is_training:True})
        print(ans.shape)
        print(np.array_equal(ans.shape, np.array([64, 10])))

Debería ver lo siguiente después de ejecutar el código de arriba:

`(64, 10)`

`True`

### GPU!
Ahora, vamos a intentar ejecutar el modelo en un dispositivo GPU, el resto del código se queda sin cambios y todas nuestras variables y operaciones serán computadas usando rutas de código acelerada. Sin embargo, si no hay GPU, obtendremos una excepción de python y tendremos que reconstruir nuestro "graph". En una CPU de dos núcleos, ejecutando el código de arriba podría ver alrededor de 50-80ms/batch, mientras que en las GPUs de Google Cloud (ejecute el código de abajo) debería ser alrededor de 2-5ms/batch.



In [ ]:
try:
    with tf.Session() as sess:
        with tf.device("/gpu:0") as dev: #"/cpu:0" or "/gpu:0"
            tf.global_variables_initializer().run()

            ans = sess.run(y_out,feed_dict={X:x,is_training:True})
            %timeit sess.run(y_out,feed_dict={X:x,is_training:True})
except tf.errors.InvalidArgumentError:
    print("no gpu found, please use Google Cloud if you want GPU acceleration")    
    # rebuild the graph
    # trying to start a GPU throws an exception 
    # and also trashes the original graph
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, [None, 32, 32, 3])
    y = tf.placeholder(tf.int64, [None])
    is_training = tf.placeholder(tf.bool)
    y_out = complex_model(X,y,is_training)

Debería observar que incluso un simple "forward pass" como este es ejecutado significativamente más rápido en GPU. Por lo tanto, para el resto de la tarea (y cuando vaya a entrenar sus modelos en la tarea 3 y su proyecto), debería usar dispositivos GPU. Sin embargo, con TensorFlow, el dispositivo por defecto es una GPU si está disponible, y una CPU en otro caso, por lo que podemos omitir la especificación del dispositivo desde ahora.

### Entrenando el Modelo.
Ahora que ha visto como definir un modelo y hacer un "forward pass" de datos por él, veamos cómo entrenaría una época completa sobre sus datos de entrenamiento (usando el "complex_model" creado más arriba).

Asegúrese que entiende como cada función de TensorFlow usada abajo corresponde a lo que usted implementó en su red neuronal personalizada.

Primero, configure un **RMSprop optimizer** (usando una taza de aprendizaje de 1e-3) y una función **cross-entropy loss**. Vea la documentación de TensorFlow para más información:

* Capas, Activaciones, Funciones de Pérdida : https://www.tensorflow.org/api_guides/python/nn
* Optimizadores: https://www.tensorflow.org/api_guides/python/train#Optimizers

In [ ]:
# Inputs
#     y_out: is what your model computes
#     y: is your TensorFlow variable with label information
# Outputs
#    mean_loss: a TensorFlow variable (scalar) with numerical loss
#    optimizer: a TensorFlow optimizer
# This should be ~3 lines of code!
mean_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(y,10), logits=y_out))
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)

In [ ]:
# batch normalization in tensorflow requires this extra dependency
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    train_step = optimizer.minimize(mean_loss)

### Entrene el Modelo
Abajo creáremos una `tf.session` y entrenaremos el modelo en una época. Debería ver una pérdida de 1.4 a 2.0 y una precisión de 0.4 a 0.5. Habrá alguna variación debido a las semillas de los números aleatorios y diferencias en inicialización.


In [ ]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())
print('Training')
run_model(sess,y_out,mean_loss,X_train,y_train,1,64,100,train_step)

### Revisando la Precisión del Modelo.
Veamos el código de entrenamiento y el código de prueba en acción -- siéntase libre de usar estos métodos cuando evalué los modelos que desarrolle abajo. Debería ver una pérdida de 1.3 a 2.0 con una precisión de 0.45 a 0.55.


In [ ]:
print('Validation')
run_model(sess,y_out,mean_loss,X_val,y_val,1,64)

## Entrenar un _gran_ modelo en CIFAR-10!

Ahora es su trabajo experimentar con arquitecturas, hiperparámetros, funciones de pérdida, y optimizadores para entrenar un modelo que logre ** >= 70% de presición en su conjunto de validación** de CIFAR-10. Puede usar la función `run_model` de arriba.


### Cosas que debería intentar:
- **Tamaño de Filtro**: Arriba usamos 7x7; Esto hace bonitas figuras, pero filtros más pequeños podrían ser más eficientes.
- **Número de Filtros**: Arriba usamos 32 filtros. ¿Más o menos funcionan mejor?
- **Convolución: Pooling vs Desnuda**: Usamos max pooling o o simplemente convoluciones desnudas?
- **Batch normalization**: Intente añadir "batch normalization" espacial después de las capas de convolución y "batch normalization" vainilla después de "affine layers". ¿Entrenan más rápido sus redes?
- **Arquitectura de la Red**: La red de arriba tiene dos capas de parámetros entrenables. ¿Se puede mejorar usando una red profunda? Buenas arquitecturas intentan incluir:
    - [conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]
    - [conv-relu-conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]
    - [batchnorm-relu-conv]xN -> [affine]xM -> [softmax or SVM]
- **Use "TensorFlow Scope"**: Use "TensorFlow scope" y/o [tf.layers](https://www.tensorflow.org/api_docs/python/tf/layers) para facilitar la escritura de redes profundas. Vea [this tutorial](https://www.tensorflow.org/tutorials/layers) para cómo usar `tf.layers`.
- **Use Decaimiento de Taza de Aprendizaje**: [Como indican las notas](http://cs231n.github.io/neural-networks-3/#anneal), decaimiento en la taza de aprendizaje puede ayudarle al modelo a converger. Cuando la pérdida no cambie en una época completa, u otra heurística que encuentre apropiada, siéntase libre de disminuir la taza. Vea la [Documentación de Tensorflow](https://www.tensorflow.org/versions/master/api_guides/python/train#Decaying_the_learning_rate) para decaimiento de taza de aprendizaje.
- **Global Average Pooling**: En lugar de aplanar y tener múltiples "affine layers", realice convoluciones hasta que la imagen se ponga pequeña (7x7 o más) y luego realice una operación de "average pooling" para obtener una imagen de 1x1 (1, 1, Filtro#), que luego se reconfigurará a un (Filtro#) vector. Esto se utiliza en [Google's Inception Network](https://arxiv.org/abs/1512.00567) (Consulte la tabla 1 para ver su arquitectura).
- **Regularización**: Agregue una regulirización de pesos L2, o quizás use [Dropout como en el tutorial TensorFlow de MNIST](https://www.tensorflow.org/get_started/mnist/pros)

### Consejos para entrenamiento
Para cada arquitectura de red que pruebe, debería ajustar la taza de entrenamiento y la intensidad de regularización. Cuando haga esto hay un par de cosas importantes que debe tener en mente:

- Si los parámetros están funcionando bien, debería ver mejora dentro de un par de cientos de iteraciones
- Recuerde una aproximación de grueso a fino para la afinación de hiperparámetros: comience por probar un largo rango de hiperparámetros durante pocas iteraciones de entrenamiento para encontrar la combinaciones de parámetros que están funcionando.
- Una vez que encuentre un conjunto de parámetros que parezca funcionar, busque más cuidadosamente cerca de estos parámetros. Puede que necesite entrenar durante más épocas.
- Debería usar el conjunto de validación para la búsqueda de hiperparámetros, y guardaremos el conjunto de prueba para evaluar su arquitectura construida en los mejores parámetros seleccionados por el conjunto de validación.


### Ir mucho más allá
Si se siente aventurero, puede implementar muchas otras funciones para intentar mejorar tu rendimiento. **No se requiere** implementar ninguno de estos; sin embargo, serían buenas cosas para intentar obtener crédito adicional.

- Pasos de actualización alternativos: para la tarea implementamos SGD+momentum, RMSprop y Adam; podría probar alternativas como AdaGrad o AdaDelta.
- Funciones de activación alternativas como leaky ReLU, ReLU paramétrica, ELU o MaxOut.
- Model ensembles
- Data augmentation
- Nuevas Arquitecturas
  - [ResNets](https://arxiv.org/abs/1512.03385) donde la entrada de la capa anterior es añadida a la salida.
  - [DenseNets](https://arxiv.org/abs/1608.06993) donde las entradas a capas previas se concatenan juntas.
  - [This blog has an in-depth overview](https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32)

Si decide implementar algo extra, describa claramente en la celda "Extra Credit Description" abajo.

### Lo que esperamos
Como mínimo, debería ser capaz de entrenar una ConvNet que obtenga **> = 70% de precisión en el conjunto de validación**. Este es solo un límite inferior: si tiene cuidado, ¡debería ser posible obtener precisiones mucho más altas que esa! Se otorgarán créditos adicionales para modelos particularmente buenos o con enfoques únicos.

Debe usar el espacio a continuación para experimentar y entrenar su red. La última celda de este notebook debe contener las precisiones del conjunto de entrenamiento y validación para su red entrenada final.

Diviértete y feliz entrenamiento!


In [ ]:
# Feel free to play with this cell

def my_model(X,y,is_training):
    
    # Conv-Relu-BN
    conv1act = tf.layers.conv2d(inputs=X, filters=32, padding='same', kernel_size=3, strides=1, activation=tf.nn.relu)
    bn1act = tf.layers.batch_normalization(inputs=conv1act, training=is_training)
    # Conv-Relu-BN
    conv2act = tf.layers.conv2d(inputs=bn1act, filters=64, padding='same', kernel_size=3, strides=1, activation=tf.nn.relu)
    bn2act = tf.layers.batch_normalization(inputs=conv2act, training=is_training)
    # Maxpool
    maxpool1act = tf.layers.max_pooling2d(inputs=bn2act, pool_size=2, strides=2)
    # Flatten
    flatten1 = tf.reshape(maxpool1act,[-1,16384])
    # FC-Relu-BN
    fc1 = tf.layers.dense(inputs=flatten1, units=1024 , activation=tf.nn.relu)
    bn3act = tf.layers.batch_normalization(inputs=fc1, training=is_training)
    # Output FC 
    y_out = tf.layers.dense(inputs=bn3act, units=10, activation=None)
    
    return y_out
    

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 32, 32, 3])
y = tf.placeholder(tf.int64, [None])
is_training = tf.placeholder(tf.bool)

y_out = my_model(X,y,is_training)

mean_loss = tf.losses.softmax_cross_entropy(logits=y_out, onehot_labels=tf.one_hot(y,10))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

# batch normalization in tensorflow requires this extra dependency
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    train_step = optimizer.minimize(mean_loss)

In [ ]:
# Feel free to play with this cell
# This default code creates a session
# and trains your model for 10 epochs
# then prints the validation set accuracy
sess = tf.Session()

sess.run(tf.global_variables_initializer())
print('Training')
run_model(sess,y_out,mean_loss,X_train,y_train,10,64,100,train_step,True)
print('Validation')
run_model(sess,y_out,mean_loss,X_val,y_val,1,64)

In [ ]:
# Test your model here, and make sure 
# the output of this cell is the accuracy
# of your best model on the training and val sets
# We're looking for >= 70% accuracy on Validation
print('Training')
run_model(sess,y_out,mean_loss,X_train,y_train,1,64)
print('Validation')
run_model(sess,y_out,mean_loss,X_val,y_val,1,64)

### Describa lo que hizo aquí
En esta celda debería escribir una explicación de lo que hizo, cualquier características adicionales que haya implementado, y cualquier visualización o "graphs" que haga en el proceso de entrnamiento y evaluación de red


Added an extra conv layer, changed all conv layers to 3x3 kernels. Added batch norm for the first fully connected layer. Change to Adam optimizer.

### Test Set - Do this only once
Now that we've gotten a result that we're happy with, we test our final model on the test set. This would be the score we would achieve on a competition. Think about how this compares to your validation set accuracy.

In [ ]:
print('Test')
run_model(sess,y_out,mean_loss,X_test,y_test,1,64)

## Going further with TensorFlow

The next assignment will make heavy use of TensorFlow. You might also find it useful for your projects. 


# Extra Credit Description
If you implement any additional features for extra credit, clearly describe them here with pointers to any code in this or other files if applicable.